# Data Preprocessing Strategy

In the last notebook, after executing our Exploratory Data Analysis, we applied a simple data preprocessing strategy and built our baseline model based on a [Ridge regressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html). In this notebook we'll focus on enhancing the preprocess pipeline to make it full use of the available data. At the end we'll check how our new features will correlate with each other and the effectiveness of the new approach by traning a new Ridge regressor model.